### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).





In [1]:
# A simple example of using PyTorch for gradient descent

import torch as t
from torch.autograd import Variable

# Define a variable, make sure requires_grad=True so that PyTorch can take gradient with respect to this variable
A = Variable(t.tensor([1.0, 1.0]), requires_grad=True)


#psat calculation 
p_water=17.4733
p_14dioxane=28.8241
x2=t.zeros(11)
p_a=t.zeros(11)
x1=t.tensor([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
p=t.tensor([28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])
for k in range(11):
    x2[k]=1-x1[k]
    p_a=((x1 * t.exp (A[0] * (((A[1] * x2) / (A[0] * x1 + A[1] * x2)))**2)*p_water) + (x2 * t.exp( (A[1] * (((A[0] * x1) / (A[0] * x1 + A[1] * x2)))**2))*p_14dioxane))

# Define a loss

loss = t.sum((p_a-p)**2)

# Take gradient
loss.backward()

# Check the gradient. numpy() turns the variable from a PyTorch tensor to a numpy array.
A.grad.numpy()


ModuleNotFoundError: No module named 'torch'

In [ ]:
# Let's examine the gradient at a different x.
A.data = t.tensor([2.0, 1.0])
loss =t.sum (((x1 * t.exp (A[0] * (((A[1] * x2) / (A[0] * x1 + A[1] * x2)))**2)*p_water + x2 * (t.exp( (A[1] * ((A[0] * x1) / (A[0] * x1 + A[1] * x2))**2))*p_14dioxane))-p)**2)

loss.backward()
A.grad.numpy()

In [ ]:
# Here is a code for gradient descent without line search

import torch as t
import numpy as np
from torch.autograd import Variable

A = Variable(t.tensor([3.0, 3.0]), requires_grad=True)

# Fix the step size
a=.001

# Start gradient descent
for i in range(10000):  # TODO: change the termination criterion

    p_a=((x1 * t.exp (A[0] * (((A[1] * x2) / (A[0] * x1 + A[1] * x2)))**2)*p_water) + (x2 * t.exp( (A[1] * (((A[0] * x1) / (A[0] * x1 + A[1] * x2)))**2))*p_14dioxane)) 
    loss = t.sum((p_a-p)**2)
    loss.backward()

    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        A -= a * A.grad
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        A.grad.zero_()

print(A.data.numpy())
print(loss.data.numpy())



In [ ]:
error_sol=t.zeros(11)
for j in range(11):
    p_a[j]=((x1[j] * t.exp ( 1.9584156* (((1.9584156 * x2[j]) / (1.9584156 * x1[j] + 1.6891813 * x2[j])))**2)*p_water) + (x2[j] * t.exp( (1.6891813 * (((1.9584156 * x1[j]) / (1.9584156 * x1[j] + 1.6891813 * x2[j])))**2))*p_14dioxane)) 
    error_sol[j]=t.abs((p_a[j]-p[j])/p[j])*100
    total_error=t.sum(error_sol)/11
print(error_sol)
print(total_error)
print(p_a)

The solution for A12 and A21 are 1.9584156 1.6891813, respectively. Comparing this to the data collected we can see that the solution is a better approximation for the outer bounds were we have a small percent error and an averaged percent error of 6.15% which is good for this type of model. Notice that no line search algrothim was used becuase the percent error is small using other techiniques might take longer to code and might not imporove the solution significanlty. 

In [ ]:
import sklearn.gaussian_process as gp
import numpy as np
from sklearn.datasets import make_classification
from matplotlib import rc
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.datasets import make_classification


%matplotlib inline

data, target = make_classification(n_samples=2500,
                                   n_features=45,
                                   n_informative=15,
                                   n_redundant=5)

def sample_loss(x):
    total = np.array([])
    for x_i in x:
        total = np.append(total, ((4 - 2.1*x[0]**2 + ((x[0]**4)/3))*x[0]**2 + x[0]*x[1] + (-4 + 4*x[1]**2)*x[1]**2) )
    
    return total + np.random.randn()

lambdas = np.linspace(1, -4, 25)
gammas = np.linspace(1, -4, 20)

# We need the cartesian combination of these two vectors
param_grid = np.array([[C, gamma] for gamma in gammas for C in lambdas])

real_loss = [sample_loss(params) for params in param_grid]

# The maximum is at:
param_grid[np.array(real_loss).argmax(), :]


def bayesian_optimization(n_iters, sample_loss, xp, yp):
  # Define the GP
    kernel = gp.kernels.Matern()
    model = gp.GaussianProcessRegressor(kernel=kernel,
                                      alpha=1e-4,
                                      n_restarts_optimizer=10,
                                      normalize_y=True)
for i in range(n_iters):
    # Update our belief of the loss function
        model.fit(xp, yp)

    # sample_next_hyperparameter is a method that computes the arg
    # max of the acquisition function
        next_sample = sample_next_hyperparameter(model, yp)

    # Evaluate the loss for the new hyperparameters
        next_loss = sample_loss(next_sample)

    # Update xp and yp
    


data, target = make_classification(n_samples=2500,
                                   n_features=45,
                                   n_informative=15,
                                   n_redundant=5)

xp, yp = bayesian_optimisation(n_iters=30, 
                               sample_loss=sample_loss, 
                               bounds=bounds,
                               n_pre_samples=3,
                               random_search=100000)
bounds = np.array([[-4, 1], [-4, 1]])


